### Test vison operator for 2 x 2 dimer model

In [1]:
import sys
import importlib
importlib.reload(sys)
if "../" not in sys.path:
    sys.path.insert(0,'../')
import numpy as np
import netket as nk
from scripts import functions as f
from scripts import new_dynamics, new_dynamics2
import os
from plotly import graph_objects as go
currentpath = os.getcwd()
parentdir = os.path.dirname(currentpath)

../scripts/functions.py:121: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.
  @jitclass(spec)


In [94]:
length=[4, 4]
V = 1.0
h = 1.0
g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)
hex_ = nk.machine.new_hex(np.array(length))

In [95]:
data = go.Scatter(
    x = hex_.lattice_coor[:,0],
    y = hex_.lattice_coor[:,1],
    text=[f'{i}' for i in range(np.prod(hex_.l) * 2)],
    mode = 'markers+text',
    textposition="top center"
)

fig = go.Figure()
fig.add_trace(
    data
)
fig.layout.yaxis = {
    'scaleanchor': 'x', 'scaleratio': 1
}
fig.layout.yaxis.showgrid = False
fig.layout.xaxis.showgrid = False
fig.show()

In [4]:
ma = nk.machine.RbmDimer(hi, hex_, alpha = 1, symmetry = True
                    ,use_hidden_bias = False, use_visible_bias = False, dtype=float, reverse=True, half=True)
ma.init_random_parameters(seed=1234)
ma._ws[:] = 0
ma._set_bare_parameters(
    ma._a, ma._b, ma._w, ma._as, ma._bs, ma._ws, ma._autom, ma._z2
)

1


In [5]:
n_jobs = 12

g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)

op = f.dimer_hamiltonian(h, V,np.array(length))
op_transition = f.dimer_flip1(length = np.array(length))
hex_ = nk.machine.new_hex(np.array(length))




sweep_size = 1000
sa_mul = nk.sampler.DimerMetropolisLocal_multi(machine=ma, op=op_transition, length = length, n_chains=1, sweep_size = sweep_size, kernel = 1, n_jobs=n_jobs)
sa_mul.reset()
_ = sa_mul.generate_samples(1000) # discard the begginings of metropolis sampling.

number of core : 12


In [77]:
n_samples_ = 10000
samples_state = sa_mul.generate_samples(int(n_samples_ / 1))
samples_state = samples_state.reshape(-1, ma.hilbert.size)
print('get samples')

get samples


In [78]:
import scripts.dynamics as Dynamics
d = Dynamics.new_dynamics(op, ma)
%time P_ = d.multiprocess(samples_state, 300, 12) 

CPU times: user 19.2 ms, sys: 132 ms, total: 151 ms
Wall time: 2.16 s


In [79]:
P = P_[0]
T = P_[1]
t_list = np.arange(0,20,0.1)
P_list = f.process_P(P, T, t_list)

In [80]:
g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)
hex_ = nk.machine.new_hex(np.array(length))

In [81]:
from importlib import reload
reload(f)
operators = f.return_vison(hi, np.arange(hex_.lattice_coor.shape[0]))

../scripts/functions.py:121: NumbaDeprecationWarning:

The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.



In [82]:
num_samples = (P_list[:,:,0]!=0).sum(axis=1)
P_list_ = P_list.reshape(-1,P_list.shape[-1])
sections1 = np.arange(P_list.shape[1])
sections2 = np.zeros(P_list_.shape[0])

_, mels1 = operators[0].get_conn_flattened(P_list[0,:,:], sections1)
sub1 = mels1.mean().real
mels1 = mels1.real

size = len(operators)

dimer_corr = np.zeros((size,t_list.shape[0]))
dimer_std = np.zeros((size,t_list.shape[0]))


for s in range(size):
    print(s)
    
    if operators[s]:
        sub2 = operators[s].get_conn_flattened(P_list[0,:,:], sections1)[1].mean().real
        _, mels2_ = operators[s].get_conn_flattened(P_list_, sections2)
        mels2 = mels2_.reshape(P_list.shape[0], P_list.shape[1]).real
        dimer_corr[s] = (mels2 * mels1).mean(axis=1)
        dimer_std[s] = (mels2 * mels1).std(axis=1)
    else:
        dimer_corr[s] = 0
        dimer_std[s] = 0

0
1
2
3
4
5
6
7


In [83]:
dimer_corr[4]

array([-4.91596639e-01, -4.87394958e-01, -4.76990796e-01, -4.65186074e-01,
       -4.57382953e-01, -4.41976791e-01, -4.21568627e-01, -3.97559024e-01,
       -3.81952781e-01, -3.60144058e-01, -3.38935574e-01, -3.23929572e-01,
       -3.01120448e-01, -2.89915966e-01, -2.76910764e-01, -2.67507003e-01,
       -2.56502601e-01, -2.37294918e-01, -2.20488195e-01, -2.01280512e-01,
       -1.91676671e-01, -1.87274910e-01, -1.78271309e-01, -1.71668667e-01,
       -1.66866747e-01, -1.59063625e-01, -1.45258103e-01, -1.42256903e-01,
       -1.41256503e-01, -1.32052821e-01, -1.27450980e-01, -1.21248499e-01,
       -1.15046018e-01, -1.02040816e-01, -9.20368147e-02, -8.92356943e-02,
       -8.58343337e-02, -8.30332133e-02, -7.96318527e-02, -7.32292917e-02,
       -6.48259304e-02, -5.54221689e-02, -5.60224090e-02, -5.54221689e-02,
       -5.58223289e-02, -5.82232893e-02, -5.06202481e-02, -4.34173669e-02,
       -3.34133653e-02, -3.48139256e-02, -3.42136855e-02, -3.04121649e-02,
       -2.96118447e-02, -

In [84]:
_, mels2_ = operators[1].get_conn_flattened(P_list_, sections2)
mels2 = mels2_.reshape(P_list.shape[0], P_list.shape[1]).real

In [85]:
samples_state[0]

array([-1., -1., -1., -1.,  1., -1.,  1., -1.])

In [86]:
op_transition.get_conn_flattened(hex_.num_to_state(240).reshape(1,-1), sections1[0].reshape(1))

(array([[ 1,  1,  1,  1, -1, -1, -1, -1],
        [ 1, -1,  1, -1, -1, -1, -1, -1],
        [ 1,  1,  1,  1, -1,  1, -1,  1]], dtype=int8),
 array([ 0.+0.j, -1.+0.j, -1.+0.j]))

## calculate exact value


$\langle 0 \vert  \sigma_0 e^{-H\tau} \sigma_0 \vert 0 \rangle$

In [87]:
H = np.zeros((8,8))

for i in range(8):
    H[i,i] = 2*V
    H[i, (i+1) % 8] = -h
    H[i, (i-1) % 8] = -h

In [88]:
v0 = np.ones(8)
v0 /= np.linalg.norm(v0)

In [89]:
from scipy.linalg import expm

v1 = np.copy(v0)
v1[1:5]  *= -1

exact_vison = []
for t in t_list:
    exact_vison.append(v1.dot(expm(-H*t)@v1))

In [90]:
(np.array(exact_vison) - dimer_corr[0])[:10]

array([-2.22044605e-16,  1.51494861e-03,  4.63935772e-03, -2.13080428e-03,
       -6.05635805e-03, -2.46469397e-03, -3.76448995e-03, -1.98088679e-03,
       -6.75089631e-04, -2.14508191e-04])

## It seems like the expmeriment works

In [91]:
v2 = np.copy(v0)
v2[4:]  *= -1

exact_vison2 = []
for t in t_list:
    exact_vison2.append(v2.dot(expm(-H*t)@v1))

In [92]:
(np.array(exact_vison2) - dimer_corr[4])[:10]

array([-0.00840336, -0.00821716, -0.00752394, -0.00391639,  0.00633149,
        0.01044805,  0.01022786,  0.00651994,  0.0109575 ,  0.00868987])

In [93]:
dimer_corr[4]

array([-4.91596639e-01, -4.87394958e-01, -4.76990796e-01, -4.65186074e-01,
       -4.57382953e-01, -4.41976791e-01, -4.21568627e-01, -3.97559024e-01,
       -3.81952781e-01, -3.60144058e-01, -3.38935574e-01, -3.23929572e-01,
       -3.01120448e-01, -2.89915966e-01, -2.76910764e-01, -2.67507003e-01,
       -2.56502601e-01, -2.37294918e-01, -2.20488195e-01, -2.01280512e-01,
       -1.91676671e-01, -1.87274910e-01, -1.78271309e-01, -1.71668667e-01,
       -1.66866747e-01, -1.59063625e-01, -1.45258103e-01, -1.42256903e-01,
       -1.41256503e-01, -1.32052821e-01, -1.27450980e-01, -1.21248499e-01,
       -1.15046018e-01, -1.02040816e-01, -9.20368147e-02, -8.92356943e-02,
       -8.58343337e-02, -8.30332133e-02, -7.96318527e-02, -7.32292917e-02,
       -6.48259304e-02, -5.54221689e-02, -5.60224090e-02, -5.54221689e-02,
       -5.58223289e-02, -5.82232893e-02, -5.06202481e-02, -4.34173669e-02,
       -3.34133653e-02, -3.48139256e-02, -3.42136855e-02, -3.04121649e-02,
       -2.96118447e-02, -